In [4]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.spatial.distance as dist
from scipy.io import wavfile
import IPython.display as ipyd
import librosa
import librosa.display
import soundfile as sf
import seaborn as sns
from matplotlib.patches import ConnectionPatch
import os
import pandas as pd
import seaborn as sns

In [5]:
   
def extract_features(filepath, filter_level):
    audio, sr = librosa.load(filepath, sr= 32000, mono=True)
    clip = librosa.effects.trim(audio, top_db= 10)
    sf.write('filtered.wav', clip[0], 32000)
    f_s, signal = wavfile.read('filtered.wav')
    n_fft = int(0.025 * f_s)  # 25 ms
    hop_length = int(0.01 * f_s)  # 10 ms
    mel_spec = librosa.feature.melspectrogram(
        y=signal / 1.0, sr=f_s, n_mels=40,
        n_fft=n_fft, hop_length=hop_length
    )
    log_mel_spec = np.log(mel_spec)
    return log_mel_spec.T

def dtw(dist_mat):

    N, M = dist_mat.shape

    # Initialize the cost matrix
    cost_mat = np.zeros((N + 1, M + 1))
    for i in range(1, N + 1):
        cost_mat[i, 0] = np.inf
    for i in range(1, M + 1):
        cost_mat[0, i] = np.inf

    # Fill the cost matrix while keeping traceback information
    traceback_mat = np.zeros((N, M))
    for i in range(N):
        for j in range(M):
            penalty = [
                cost_mat[i, j],  # match (0)
                cost_mat[i, j + 1],  # insertion (1)
                cost_mat[i + 1, j]]  # deletion (2)
            i_penalty = np.argmin(penalty)
            cost_mat[i + 1, j + 1] = dist_mat[i, j] + penalty[i_penalty]
            traceback_mat[i, j] = i_penalty

    # Traceback from bottom right
    i = N - 1
    j = M - 1
    path = [(i, j)]
    while i > 0 or j > 0:
        tb_type = traceback_mat[i, j]
        if tb_type == 0:
            # Match
            i = i - 1
            j = j - 1
        elif tb_type == 1:
            # Insertion
            i = i - 1
        elif tb_type == 2:
            # Deletion
            j = j - 1
        path.append((i, j))

    # Strip infinity edges from cost_mat before returning
    cost_mat = cost_mat[1:, 1:]
    return path[::-1], cost_mat

def getCost(x_seq, y_seq):
    M = y_seq.shape[0]
    N = x_seq.shape[0]
    dist_mat = dist.cdist(x_seq, y_seq, "cosine")
    path, cost_mat = dtw(dist_mat)
    cost_mat_normalized = cost_mat / (M + N)

    return path, cost_mat, cost_mat_normalized

def splitSegments(seg_length, y, step_size):
    step_num = 0
    y_length = y.shape[0]
    y_segments = []
    while (step_num*step_size+seg_length <= y_length):
        y_segments.append(y[step_num*step_size:step_num*step_size+seg_length])
        step_num+=1
    return y_segments

def splitSearch(q, search, step_size):
    q_length = q.shape[0]
    search_length = search.shape[0]
    step_num = 0
    searches = []
    while (step_num*step_size+q_length < search_length):
        searches.append(search[step_num*step_size:step_num*step_size+q_length])
        step_num+=1
    if step_num == 0:
        searches.append(search[step_num*step_size:step_num*step_size+q_length])
    return searches

def segmental_cost_mat(x_seg, y_seg):
    N=len(x_seg)
    M=len(y_seg)
    cost_mat = np.zeros((N, M))
    for i in range(N):
        for j in range(M):
            path, cost_mat_non_norm, cost_mat_normalized = getCost(x_seg[i], y_seg[j])
            cost_mat[i, j] = 1/(cost_mat_normalized[-1, -1].tolist())
    return cost_mat

def k_nearest(cost_mat, k):
    nearest_max=[]
    for i in range(k):
        max_index = np.where(cost_mat == np.amax(cost_mat))
        max_index = list(zip(max_index[0], max_index[1]))
        cost_mat[max_index[0][0],max_index[0][1]] = 0
        nearest_max.append(max_index[0])
    return nearest_max

def nearest(cost_mat):
    max_index = np.where(cost_mat == np.amax(cost_mat))
    max_index = list(zip(max_index[0], max_index[1]))
    print(max_index)
    return max_index

def identified_segment(x_seg, y_seg, nearest_max, k):
    i = nearest_max[k][0]
    j = nearest_max[k][1]
    x_kseg = x[step_num*step_size:step_num*step_size+seg_length]
    
def getScaled(num_costs, num):
    max_cost_len=max(num_costs)
    if num_costs[num] > 1:
        div=max_cost_len/float(num_costs[num]-1)
        x=[0]
        for i in range(1,num_costs[num]):
            x.append(i*div)
    else: 
        x=[max_cost_len/2]
    return x

def getCosts(q, searches):
    costs=[]
    for i in searches:
        path, cost_mat, cost_mat_normalized = getCost(q, i)
        costs.append(cost_mat_normalized[-1, -1].tolist())
    return costs
